<a href="https://colab.research.google.com/github/VellummyilumVinoth/Train_And_Test_Using_ALBERT/blob/main/FinetunedAlbert.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers

!pip install sentencepiece

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 94.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.8/199.8 KB 27.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 81.6 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 63.7 MB/s eta 0:00:00


In [2]:
from transformers import AlbertForMaskedLM, AlbertTokenizer

# Load the ALBERT tokenizer and model
tokenizer = AlbertTokenizer.from_pretrained('albert-base-v2')
model = AlbertForMaskedLM.from_pretrained('albert-base-v2')

In [3]:
import csv

variable_names = []
statements = []

with open('output.csv', 'r') as file:
    reader = csv.reader(file)
    for row in reader:
        # Check if the row contains at least two columns
        if len(row) >= 2:
            # Append the variable name and source statement to their respective lists
            variable_names.append(row[0])
            statements.append(row[1])

# Convert the list of variable names into a set to remove duplicates
variable_names_set = set(variable_names)

# Tokenize the variable names and add them to the tokenizer's vocabulary
variable_name_tokens = tokenizer.tokenize(' '.join(variable_names_set))

# Add the variable names to the tokenizer's vocabulary
for token in variable_name_tokens:
    # Check if the token is not already in the vocabulary before adding it
    if token not in tokenizer.get_vocab():
        tokenizer.add_tokens(token)


In [4]:
variable_names

['headers',
 'message',
 'payload',
 'message',
 'payload',
 'message',
 'payload',
 'message',
 'payload',
 'message',
 'payload',
 'message',
 'payload',
 'message',
 'payload',
 'message',
 'payload',
 'message',
 'payload',
 'payload',
 'message',
 'payload',
 'message',
 'payload',
 'message',
 'payload',
 'message',
 'payload',
 'message',
 'payload',
 'textPrompt',
 'completionRes',
 'imagePrmt',
 'imageRes',
 'product',
 'prodObj',
 'pid',
 'lastRowIndex',
 'row',
 'message',
 'payload',
 'message',
 'payload',
 'message',
 'payload',
 'outputStream',
 'streamValue',
 'nextRecord',
 'clientEndpoint',
 'returnArray',
 'count',
 'data',
 'variable',
 'dbClient1',
 'dbClient2',
 'dbClient3',
 'dbClient4',
 'dbClient5',
 'resultArray',
 'i',
 'value',
 'sqlError',
 'sqlError',
 'rwType',
 'roType',
 'roBdd',
 'rwBdd',
 's',
 'i',
 'dummy',
 'sortedFields',
 'names',
 'types',
 'b',
 'mm',
 'm',
 'res',
 'isEmpty',
 'combined',
 'p',
 'm',
 'neg',
 'pairing',
 'd',
 'mt',
 'i',
 'po

In [5]:
statements

['map<string|string[]> headers = {};',
 'AddItemRequest message;',
 'var payload = check self.grpcClient->executeSimpleRPC(\\hipstershop.CartService/AddItem\\"',
 'GetCartRequest message;',
 'var payload = check self.grpcClient->executeSimpleRPC(\\hipstershop.CartService/GetCart\\"',
 'EmptyCartRequest message;',
 'var payload = check self.grpcClient->executeSimpleRPC(\\hipstershop.CartService/EmptyCart\\"',
 'ListRecommendationsRequest message;',
 'var payload = check self.grpcClient->executeSimpleRPC(\\hipstershop.RecommendationService/ListRecommendations\\"',
 'Empty message;',
 'var payload = check self.grpcClient->executeSimpleRPC(\\hipstershop.ProductCatalogService/ListProducts\\"',
 'GetProductRequest message;',
 'var payload = check self.grpcClient->executeSimpleRPC(\\hipstershop.ProductCatalogService/GetProduct\\"',
 'SearchProductsRequest message;',
 'var payload = check self.grpcClient->executeSimpleRPC(\\hipstershop.ProductCatalogService/SearchProducts\\"',
 'GetQuoteReques

In [6]:
import re

def clean_text(line):
    line = re.sub(r'-+',' ',line)
    line = re.sub(r'[^a-zA-Z, ]+'," ",line)
    line = re.sub(r'[ ]+'," ",line)
    line += ";"
    return line

source_statements = []
len_lst = []
for line in statements:
    if len(line.split(" ")) >=0:
        line = clean_text(line)
        source_statements.append(line)
        len_lst.append(len(line.split(" ")))


In [7]:
source_statements

['map string string headers ',
 'AddItemRequest message ',
 'var payload check self grpcClient executeSimpleRPC hipstershop CartService AddItem ',
 'GetCartRequest message ',
 'var payload check self grpcClient executeSimpleRPC hipstershop CartService GetCart ',
 'EmptyCartRequest message ',
 'var payload check self grpcClient executeSimpleRPC hipstershop CartService EmptyCart ',
 'ListRecommendationsRequest message ',
 'var payload check self grpcClient executeSimpleRPC hipstershop RecommendationService ListRecommendations ',
 'Empty message ',
 'var payload check self grpcClient executeSimpleRPC hipstershop ProductCatalogService ListProducts ',
 'GetProductRequest message ',
 'var payload check self grpcClient executeSimpleRPC hipstershop ProductCatalogService GetProduct ',
 'SearchProductsRequest message ',
 'var payload check self grpcClient executeSimpleRPC hipstershop ProductCatalogService SearchProducts ',
 'GetQuoteRequest message ',
 'var payload check self grpcClient executeS

In [8]:
from huggingface_hub import add_space_secret
import pandas as pd
import torch
import os
import re
from transformers import AdamW, Trainer, TrainingArguments, DataCollatorForLanguageModeling

import random

class VariableNamesDataset(torch.utils.data.Dataset):
    def __init__(self, variable_names, source_statements, tokenizer, mask_probability):
        self.variable_names = variable_names
        self.source_statements = source_statements
        self.tokenizer = tokenizer
        self.mask_probability = mask_probability

    def __len__(self):
        return len(self.source_statements)
    
    def __getitem__(self, idx):
        variable_name = str(self.variable_names[idx])
        source_statement = str(self.source_statements[idx])

        print("variable names: ",len(variable_name))
        print("source statements: ",len(source_statement))

        # Tokenize the variable name and source statement
        variable_name_tokens = self.tokenizer.tokenize(variable_name)
        source_statement_tokens = self.tokenizer.tokenize(source_statement)

        print("Variable name tokens:", variable_name_tokens)
        print("Source statement tokens:", source_statement_tokens)

        # Select a variable name tokens to mask
        variable_name_indices = [i for i, token in enumerate(source_statement_tokens) if token in variable_name_tokens]

        print("Variable name indices:", variable_name_indices)

        num_to_mask = int(len(variable_name_indices)* self.mask_probability)
        indices_to_mask = random.sample(variable_name_indices, num_to_mask)

        print("Num to mask:", num_to_mask)
        print("Indices to mask:", indices_to_mask)

        # Replace the selected variable name tokens with the [MASK] token
        for i in indices_to_mask:
            source_statement_tokens[i] = '[MASK]'

        # Convert the tokens back to text
        masked_source_statement = self.tokenizer.convert_tokens_to_string(source_statement_tokens)

        # Tokenize the masked source statement
        input_ids = self.tokenizer.encode(
            masked_source_statement,
            add_special_tokens=False,
            truncation=True,
            padding='max_length',
            max_length=128
        )

        print("Input IDs:", input_ids)
        print("Masked source statement:", masked_source_statement)

        return torch.tensor(input_ids)
 

In [9]:
# Split the data into train and test sets
train_size = int(len(source_statements) * 0.9)
train_variable_names = variable_names[:train_size]
train_source_statements = source_statements[:train_size]
test_variable_names = variable_names[train_size:]
test_source_statements = source_statements[train_size:]

In [10]:
train_size

2187

In [11]:
# Prepare the data for training and testing
train_dataset = VariableNamesDataset(train_variable_names, train_source_statements, tokenizer, mask_probability = 1)

test_dataset = VariableNamesDataset(test_variable_names, test_source_statements, tokenizer, mask_probability = 0)

In [12]:
train_dataset[7]

variable names:  7
source statements:  35
Variable name tokens: ['▁message']
Source statement tokens: ['▁list', 're', 'com', 'men', 'd', 'ations', 're', 'quest', '▁message']
Variable name indices: [8]
Num to mask: 1
Indices to mask: [8]
Input IDs: [968, 99, 960, 755, 43, 7504, 99, 10351, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
Masked source statement: listrecommendationsrequest[MASK]


tensor([  968,    99,   960,   755,    43,  7504,    99, 10351,     4,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0])

In [13]:
test_dataset[0]

variable names:  1
source statements:  28
Variable name tokens: ['▁f']
Source statement tokens: ['▁future', '▁not', 'ify', 'result', '▁error', '▁f']
Variable name indices: [5]
Num to mask: 0
Indices to mask: []
Input IDs: [1022, 52, 8612, 29955, 7019, 398, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
Masked source statement: future notifyresult error f


tensor([ 1022,    52,  8612, 29955,  7019,   398,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0])

In [14]:
# Prepare the data collator
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer, mlm=True, mlm_probability=0.15
)

# Set up training arguments
training_args = TrainingArguments(
    output_dir='./results',          # output directory
    evaluation_strategy = "steps",   # Evaluation and Save happens every X steps
    eval_steps = 50,                # Evaluation happens every X steps
    save_steps = 1000,               # Save checkpoint every X steps
    num_train_epochs = 10,            # Total number of training epochs
    learning_rate = 1e-5,            # Learning rate
    per_device_train_batch_size=32,  # Batch size per device during training
    per_device_eval_batch_size=64,   # Batch size for evaluation
    warmup_steps = 500,              # Number of warmup steps
    weight_decay = 0.01,             # Strength of weight decay
    logging_dir='./logs',            # Directory for storing logs
    logging_steps=1000,              # Log every X steps
)

# Instantiate the Trainer class and train the model
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    data_collator=data_collator,
)

trainer.train()


/usr/local/lib/python3.9/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


variable names:  9
source statements:  18
Variable name tokens: ['▁cons', 't', 'con', 'd']
Source statement tokens: ['▁boo', 'le', 'an', '▁cons', 't', 'con', 'd']
Variable name indices: [3, 4, 5, 6]
Num to mask: 4
Indices to mask: [3, 6, 4, 5]
Input IDs: [9827, 413, 210, 4, 4, 4, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
Masked source statement: boolean[MASK][MASK][MASK][MASK]
variable names:  9
source statements:  38
Variable name tokens: ['▁min', 'length']
Source statement tokens: ['▁in', 't', '▁min', 'length', '▁pos', '▁members', '▁fixed', 'length']
Variable name indices: [2, 3, 7]
Num to mask: 3
Indices to mask: [2, 7, 3]
Input IDs: [19, 38, 4, 4, 12928, 443, 3535

Step,Training Loss,Validation Loss
50,No log,14.933205
100,No log,13.770005
150,No log,9.415291
200,No log,7.885166
250,No log,7.718249
300,No log,8.012436
350,No log,7.675022
400,No log,7.558167
450,No log,7.495205
500,No log,7.604706


Streaming output truncated to the last 5000 lines.
Variable name indices: [1, 2, 3, 4, 5, 6]
Num to mask: 6
Indices to mask: [2, 5, 6, 3, 4, 1]
Input IDs: [8719, 4, 4, 4, 4, 4, 4, 9647, 96, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
Masked source statement: bir[MASK][MASK][MASK][MASK][MASK][MASK]arith
variable names:  4
source statements:  42
Variable name tokens: ['▁kind']
Source statement tokens: ['▁in', 't', '▁kind', '▁emission', 'kind', 'to', 'int', '▁get', '▁prop', 's', '▁kind']
Variable name indices: [2, 10]
Num to mask: 2
Indices to mask: [10, 2]
Input IDs: [19, 38, 4, 18609, 11008, 262, 6391, 164, 9476, 18, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

TrainOutput(global_step=690, training_loss=7.89713966259058, metrics={'train_runtime': 645.2139, 'train_samples_per_second': 33.896, 'train_steps_per_second': 1.069, 'total_flos': 122878825390080.0, 'train_loss': 7.89713966259058, 'epoch': 10.0})

In [15]:
# Save the trained model and tokenizer
output_dir = './finetuned_albert'
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

model.save_pretrained(output_dir)
tokenizer.save_pretrained(output_dir)


('./finetuned_albert/tokenizer_config.json',
 './finetuned_albert/special_tokens_map.json',
 './finetuned_albert/spiece.model',
 './finetuned_albert/added_tokens.json')

In [24]:
import torch
from transformers import AlbertTokenizer, AlbertForMaskedLM
from tabulate import tabulate

# Load the trained model and tokenizer
output_dir = './finetuned_albert'
model = AlbertForMaskedLM.from_pretrained(output_dir)
tokenizer = AlbertTokenizer.from_pretrained(output_dir)

# Define a sample masked statement
masked_statement = "string [MASK] = self.curName1();"

# Tokenize the masked statement
input_ids = tokenizer.encode(masked_statement, add_special_tokens=True, return_tensors='pt')

# Find the position of the masked token
masked_token_index = torch.where(input_ids == tokenizer.mask_token_id)[1][0].item()

# Generate predictions for the masked token using the fine-tuned model
with torch.no_grad():
    outputs = model(input_ids)
    predictions = outputs[0]

# Get the top 5 predictions and their probability scores from the fine-tuned model
probs_ft = torch.nn.functional.softmax(predictions[0, masked_token_index], dim=-1)
top_k_ft = torch.topk(probs_ft, k=5)

# Generate predictions for the masked token using the base model
model_base = AlbertForMaskedLM.from_pretrained('albert-base-v2')
tokenizer_base = AlbertTokenizer.from_pretrained('albert-base-v2')
input_ids_base = tokenizer_base.encode(masked_statement, add_special_tokens=True, return_tensors='pt')
with torch.no_grad():
    outputs = model_base(input_ids_base)
    predictions = outputs[0]

# Get the top 5 predictions and their probability scores from the base model
probs_base = torch.nn.functional.softmax(predictions[0, masked_token_index], dim=-1)
top_k_base = torch.topk(probs_base, k=5)

# Create a table with the top predictions and their probabilities from both models
table = [["Fine-Tuned Model", f"{tokenizer.mask_token}"] + [tokenizer.convert_ids_to_tokens([idx])[0] for idx in top_k_ft.indices],
         ["Probability", ""] + [f"{probs_ft[idx].item():.4f}" for idx in top_k_ft.indices],
         ["Base Model", f"{tokenizer_base.mask_token}"] + [tokenizer_base.convert_ids_to_tokens([idx])[0] for idx in top_k_base.indices],
         ["Probability", ""] + [f"{probs_base[idx].item():.4f}" for idx in top_k_base.indices]]

# Print the table
print(tabulate(table, headers="firstrow", tablefmt="fancy_grid"))


╒════════════════════╤══════════╤═══════════════╤═════════════╤═════════╤══════════════╤════════╕
│ Fine-Tuned Model   │ [MASK]   │ ▁identifier   │ type        │ value   │ ▁parameter   │ data   │
╞════════════════════╪══════════╪═══════════════╪═════════════╪═════════╪══════════════╪════════╡
│ Probability        │          │ 0.0930        │ 0.0645      │ 0.0375  │ 0.0301       │ 0.0274 │
├────────────────────┼──────────┼───────────────┼─────────────┼─────────┼──────────────┼────────┤
│ Base Model         │ [MASK]   │ ++            │ ▁identifier │ pmid    │ ://          │ =      │
├────────────────────┼──────────┼───────────────┼─────────────┼─────────┼──────────────┼────────┤
│ Probability        │          │ 0.1948        │ 0.0570      │ 0.0434  │ 0.0240       │ 0.0238 │
╘════════════════════╧══════════╧═══════════════╧═════════════╧═════════╧══════════════╧════════╛


In [22]:
import torch
from transformers import AlbertTokenizer, AlbertForMaskedLM

# Load the trained model and tokenizer
output_dir = './finetuned_albert'
finetuned_model = AlbertForMaskedLM.from_pretrained(output_dir)
finetuned_tokenizer = AlbertTokenizer.from_pretrained(output_dir)

# Load the ALBERT model and tokenizer
base_model = AlbertForMaskedLM.from_pretrained('albert-base-v2')
base_tokenizer = AlbertTokenizer.from_pretrained('albert-base-v2')

# Define a list of sample masked statements
masked_statements = ["int [MASK] = getCount();", "Student [MASK];", "Person [MASK] = {name: 'John'};", "int[] [MASK] = getNumbers();", "Person[] [MASK] = getPersons();", "Person[] [MASK] = getPersons(10, 'Sales');"]

# Loop through each masked statement and generate predictions for both models
for masked_statement in masked_statements:
    print(f"Masked statement: {masked_statement}\n")
    # Tokenize the masked statement for the fine-tuned model
    input_ids = finetuned_tokenizer.encode(masked_statement, add_special_tokens=True, return_tensors='pt')
    masked_token_index = torch.where(input_ids == finetuned_tokenizer.mask_token_id)[1][0].item()
    with torch.no_grad():
        outputs = finetuned_model(input_ids)
        predictions = outputs[0]
    probs = torch.nn.functional.softmax(predictions[0, masked_token_index], dim=-1)
    top_k = torch.topk(probs, k=5)
    # Print the top 5 predictions and their probability scores for the fine-tuned model
    print("Fine-tuned model predictions:")
    print("{:<20} {:<20}".format('Prediction', 'Probability'))
    for i, idx in enumerate(top_k.indices):
        token = finetuned_tokenizer.convert_ids_to_tokens([idx])[0]
        prob = top_k.values[i].item()
        print("{:<20} {:.4f}".format(token, prob))
    print("\n")
    
    # Tokenize the masked statement for the base model
    input_ids = base_tokenizer.encode(masked_statement, add_special_tokens=True, return_tensors='pt')
    masked_token_index = torch.where(input_ids == base_tokenizer.mask_token_id)[1][0].item()
    with torch.no_grad():
        outputs = base_model(input_ids)
        predictions = outputs[0]
    probs = torch.nn.functional.softmax(predictions[0, masked_token_index], dim=-1)
    top_k = torch.topk(probs, k=5)
    # Print the top 5 predictions and their probability scores for the base model
    print("Base model predictions:")
    print("{:<20} {:<20}".format('Prediction', 'Probability'))
    for i, idx in enumerate(top_k.indices):
        token = base_tokenizer.convert_ids_to_tokens([idx])[0]
        prob = top_k.values[i].item()
        print("{:<20} {:.4f}".format(token, prob))
    print("\n")


Masked statement: int [MASK] = getCount();

Fine-tuned model predictions:
Prediction           Probability         
value                0.0453
type                 0.0362
data                 0.0361
text                 0.0199
▁value               0.0163


Base model predictions:
Prediction           Probability         
%                    0.0497
koppen               0.0392
=                    0.0268
#                    0.0228
rq                   0.0227


Masked statement: Student [MASK];

Fine-tuned model predictions:
Prediction           Probability         
▁obligation          0.0758
▁requirements        0.0432
▁values              0.0221
▁text                0.0218
▁request             0.0208


Base model predictions:
Prediction           Probability         
▁eireann             0.0281
▁loans               0.0267
▁accreditation       0.0181
▁essays              0.0105
▁affairs             0.0095


Masked statement: Person [MASK] = {name: 'John'};

Fine-tuned model predictio